In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [2]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


#Import statements

In [3]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (DataCollatorWithPadding,create_optimizer,DebertaTokenizerFast)

# **Data preparation**

In [4]:
dataset = load_dataset("covid_qa_deepset")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for covid_qa_deepset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/covid_qa_deepset
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 2019
    })
})

#Configuration's parameters

In [6]:
BATCH_SIZE = 16
MAX_LENGTH = 512

In [7]:
dataset['train'][0]

{'document_id': 630,
 'context': "Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child Transmission of HIV-1\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC2752805/\n\nBoily-Larouche, Geneviève; Iscache, Anne-Laure; Zijenah, Lynn S.; Humphrey, Jean H.; Mouland, Andrew J.; Ward, Brian J.; Roger, Michel\n2009-10-07\nDOI:10.1371/journal.pone.0007211\nLicense:cc-by\n\nAbstract: BACKGROUND: Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide. Given that the C-type lectin receptor, dendritic cell-specific ICAM-grabbing non-integrin-related (DC-SIGNR, also known as CD209L or liver/lymph node–specific ICAM-grabbing non-integrin (L-SIGN)), can interact with pathogens including HIV-1 and is expressed at the maternal-fetal interface, we hypothesized that it could influence MTCT of HIV-1. METHODS AND FINDINGS: To investigate the potential role of DC-SIGNR in MTCT of HIV-1, we carried out a genetic association study of DC-SIGNR in a

In [8]:
answer='Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide.'
print(len(answer)+370)

465


In [9]:
dataset['train'][1]

{'document_id': 630,
 'context': "Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child Transmission of HIV-1\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC2752805/\n\nBoily-Larouche, Geneviève; Iscache, Anne-Laure; Zijenah, Lynn S.; Humphrey, Jean H.; Mouland, Andrew J.; Ward, Brian J.; Roger, Michel\n2009-10-07\nDOI:10.1371/journal.pone.0007211\nLicense:cc-by\n\nAbstract: BACKGROUND: Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide. Given that the C-type lectin receptor, dendritic cell-specific ICAM-grabbing non-integrin-related (DC-SIGNR, also known as CD209L or liver/lymph node–specific ICAM-grabbing non-integrin (L-SIGN)), can interact with pathogens including HIV-1 and is expressed at the maternal-fetal interface, we hypothesized that it could influence MTCT of HIV-1. METHODS AND FINDINGS: To investigate the potential role of DC-SIGNR in MTCT of HIV-1, we carried out a genetic association study of DC-SIGNR in a

In [10]:
answer='DC-SIGNR plays a crucial role in MTCT of HIV-1 and that impaired placental DC-SIGNR expression increases risk of transmission.'
print(len(answer)+370)

496


In [11]:
from transformers import LongformerTokenizerFast, TFLongformerForQuestionAnswering
model_id="allenai/longformer-large-4096-finetuned-triviaqa"


In [12]:
tokenizer=LongformerTokenizerFast.from_pretrained(model_id)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

In [13]:
tokenizer

LongformerTokenizerFast(name_or_path='allenai/longformer-large-4096-finetuned-triviaqa', vocab_size=50265, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [14]:
tokenized_examples = tokenizer(
    dataset['train'][0]['question'],
    dataset['train'][0]["context"],
    truncation="only_second",
    max_length=MAX_LENGTH,
    stride=64,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

In [15]:
len(tokenized_examples["input_ids"])

18

In [16]:
list_token=tokenizer.tokenize("[CLS]What is the main cause of HIV-1 infection in children?[SEP]Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child Transmission of HIV-1\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC2752805/\n\nBoily-Larouche, Geneviève; Iscache, Anne-Laure; Zijenah, Lynn S.; Humphrey, Jean H.; Mouland, Andrew J.; Ward, Brian J.; Roger, Michel\n2009-10-07\nDOI:10.1371/journal.pone.0007211\nLicense:cc-by\n\nAbstract: BACKGROUND: Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide.")
print(list_token)
for i in range(len(list_token)):
  if list_token[i]=='Ġchildren':
    print(i)

['[', 'CL', 'S', ']', 'What', 'Ġis', 'Ġthe', 'Ġmain', 'Ġcause', 'Ġof', 'ĠHIV', '-', '1', 'Ġinfection', 'Ġin', 'Ġchildren', '?', '[', 'SE', 'P', ']', 'Function', 'al', 'ĠGenetic', 'ĠVari', 'ants', 'Ġin', 'ĠDC', '-', 'SIGN', 'R', 'ĠAre', 'ĠAssociated', 'Ġwith', 'ĠMother', '-', 'to', '-', 'Child', 'ĠTransmission', 'Ġof', 'ĠHIV', '-', '1', 'Ċ', 'Ċ', 'https', '://', 'www', '.', 'nc', 'bi', '.', 'nl', 'm', '.', 'nih', '.', 'gov', '/', 'pm', 'c', '/', 'articles', '/', 'PM', 'C', '27', '52', '805', '/', 'Ċ', 'Ċ', 'Bo', 'ily', '-', 'L', 'ar', 'ou', 'che', ',', 'ĠGene', 'vi', 'Ã¨', 've', ';', 'ĠIs', 'cache', ',', 'ĠAnne', '-', 'La', 'ure', ';', 'ĠZ', 'ij', 'en', 'ah', ',', 'ĠLynn', 'ĠS', '.;', 'ĠHumph', 'rey', ',', 'ĠJean', 'ĠH', '.;', 'ĠMou', 'land', ',', 'ĠAndrew', 'ĠJ', '.;', 'ĠWard', ',', 'ĠBrian', 'ĠJ', '.;', 'ĠRoger', ',', 'ĠMichel', 'Ċ', '2009', '-', '10', '-', '07', 'Ċ', 'DO', 'I', ':', '10', '.', '13', '71', '/', 'journal', '.', 'p', 'one', '.', '000', '72', '11', 'Ċ', 'License', ':', '

In [17]:
tokenizer.encode("What is the main cause of HIV-1 infection in children?")

[0, 2264, 16, 5, 1049, 1303, 9, 7947, 12, 134, 7910, 11, 408, 116, 2]

In [18]:
for ids in tokenized_examples["input_ids"]:
  print(ids)
  print('-->',tokenizer.decode(ids))

[0, 2264, 16, 5, 1049, 1303, 9, 7947, 12, 134, 7910, 11, 408, 116, 2, 2, 47802, 337, 42215, 41058, 3277, 11, 5815, 12, 42101, 500, 3945, 1562, 19, 8133, 12, 560, 12, 39487, 33308, 9, 7947, 12, 134, 50118, 50118, 13082, 640, 1401, 4, 11326, 5605, 4, 35760, 119, 4, 24469, 4, 7354, 73, 1685, 438, 73, 32315, 73, 5683, 347, 2518, 4429, 34249, 73, 50118, 50118, 18935, 5846, 12, 574, 271, 1438, 2871, 6, 13120, 6873, 8025, 548, 131, 1534, 47974, 6, 7896, 12, 10766, 2407, 131, 525, 2161, 225, 895, 6, 9840, 208, 10116, 16788, 5460, 6, 5363, 289, 10116, 26805, 1245, 6, 2224, 344, 10116, 5986, 6, 2995, 344, 10116, 6682, 6, 11026, 50118, 23301, 12, 698, 12, 3570, 50118, 19174, 100, 35, 698, 4, 1558, 5339, 73, 15545, 4, 642, 1264, 4, 151, 4956, 1225, 50118, 48407, 35, 7309, 12, 1409, 50118, 50118, 47638, 35, 23098, 45604, 35, 8133, 12, 560, 12, 14069, 9235, 36, 11674, 7164, 43, 16, 5, 1049, 1303, 9, 7947, 12, 134, 7910, 11, 408, 3612, 4, 6211, 14, 5, 230, 12, 12528, 25673, 179, 33915, 6, 385, 1397, 

In [19]:
offset_mapping_list=[(0, 4), (4, 7), (7, 11), (11, 16), (16, 22), (22, 25), (25, 29), (29, 30), (30, 31), (31, 41), (41, 44), (44, 53), (53, 54), (0, 0), (0, 8), (8, 10), (10, 18), (18, 23), (23, 27), (27, 30), (30, 33), (33, 34), (34, 38), (38, 39), (39, 43), (43, 54), (54, 59), (59, 66), (66, 67), (67, 69), (69, 70), (70, 75), (75, 88), (88, 91), (91, 95), (95, 96), (96, 97), (97, 98), (98, 99), (99, 104), (104, 107), (107, 110), (110, 111), (111, 113), (113, 115), (115, 116), (116, 118), (118, 119), (119, 120), (120, 123), (123, 124), (124, 127), (127, 128), (128, 130), (130, 131), (131, 132), (132, 140), (140, 141), (141, 143), (143, 144), (144, 146), (146, 148), (148, 151), (151, 152), (152, 153), (153, 154), (154, 156), (156, 159), (159, 160), (160, 161), (161, 163), (163, 165), (165, 168), (168, 169), (169, 174), (174, 176), (176, 177), (177, 179), (179, 180), (180, 183), (183, 188), (188, 189), (189, 194), (194, 195), (195, 197), (197, 200), (200, 201), (201, 203), (203, 205), (205, 207), (207, 209), (209, 210), (210, 215), (215, 217), (217, 219), (219, 225), (225, 228), (228, 229), (229, 234), (234, 236), (236, 238), (238, 242), (242, 246), (246, 247), (247, 254), (254, 256), (256, 258), (258, 263), (263, 264), (264, 270), (270, 272), (272, 274), (274, 280), (280, 281), (281, 288), (288, 289), (289, 293), (293, 294), (294, 296), (296, 297), (297, 299), (299, 300), (300, 302), (302, 303), (303, 304), (304, 306), (306, 307), (307, 309), (309, 311), (311, 312), (312, 319), (319, 320), (320, 321), (321, 324), (324, 325), (325, 328), (328, 330), (330, 332), (332, 333), (333, 340), (340, 341), (341, 343), (343, 344), (344, 346), (346, 347), (347, 348), (348, 356), (356, 357), (357, 362), (362, 368), (368, 369), (369, 376),]
print(len(offset_mapping_list))

152


In [20]:
tokenized_examples

{'input_ids': [[0, 2264, 16, 5, 1049, 1303, 9, 7947, 12, 134, 7910, 11, 408, 116, 2, 2, 47802, 337, 42215, 41058, 3277, 11, 5815, 12, 42101, 500, 3945, 1562, 19, 8133, 12, 560, 12, 39487, 33308, 9, 7947, 12, 134, 50118, 50118, 13082, 640, 1401, 4, 11326, 5605, 4, 35760, 119, 4, 24469, 4, 7354, 73, 1685, 438, 73, 32315, 73, 5683, 347, 2518, 4429, 34249, 73, 50118, 50118, 18935, 5846, 12, 574, 271, 1438, 2871, 6, 13120, 6873, 8025, 548, 131, 1534, 47974, 6, 7896, 12, 10766, 2407, 131, 525, 2161, 225, 895, 6, 9840, 208, 10116, 16788, 5460, 6, 5363, 289, 10116, 26805, 1245, 6, 2224, 344, 10116, 5986, 6, 2995, 344, 10116, 6682, 6, 11026, 50118, 23301, 12, 698, 12, 3570, 50118, 19174, 100, 35, 698, 4, 1558, 5339, 73, 15545, 4, 642, 1264, 4, 151, 4956, 1225, 50118, 48407, 35, 7309, 12, 1409, 50118, 50118, 47638, 35, 23098, 45604, 35, 8133, 12, 560, 12, 14069, 9235, 36, 11674, 7164, 43, 16, 5, 1049, 1303, 9, 7947, 12, 134, 7910, 11, 408, 3612, 4, 6211, 14, 5, 230, 12, 12528, 25673, 179, 33915,

In [21]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
offset_mapping = tokenized_examples.pop("offset_mapping")

tokenized_examples["start_positions"] = []#[152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
tokenized_examples["end_positions"] = []#[172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [22]:
for i, offsets in enumerate(offset_mapping):
  if len(dataset['train'][0]['answers']['answer_start'])==0:
    tokenized_examples["start_positions"].append(0)
    tokenized_examples["end_positions"].append(0)
  else:

    start_char=dataset['train'][0]['answers']['answer_start'][0]
    end_char=start_char+len(dataset['train'][0]['answers']['text'][0])
    found=0
    start_token_position=0
    end_token_position=0

    for j,offset in enumerate(offsets):
      if offset[0]<=start_char and offset[1]>=start_char and found==0:
        start_token_position=j
        end_token_position=MAX_LENGTH

        found=1
      if offset[1]>=end_char and found==1:
        end_token_position=j
        break
    tokenized_examples["start_positions"].append(start_token_position)
    tokenized_examples["end_positions"].append(end_token_position)

In [23]:
print(tokenized_examples['start_positions'])
print(tokenized_examples['end_positions'])

[153, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[175, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
print(tokenized_examples['input_ids'])

[[0, 2264, 16, 5, 1049, 1303, 9, 7947, 12, 134, 7910, 11, 408, 116, 2, 2, 47802, 337, 42215, 41058, 3277, 11, 5815, 12, 42101, 500, 3945, 1562, 19, 8133, 12, 560, 12, 39487, 33308, 9, 7947, 12, 134, 50118, 50118, 13082, 640, 1401, 4, 11326, 5605, 4, 35760, 119, 4, 24469, 4, 7354, 73, 1685, 438, 73, 32315, 73, 5683, 347, 2518, 4429, 34249, 73, 50118, 50118, 18935, 5846, 12, 574, 271, 1438, 2871, 6, 13120, 6873, 8025, 548, 131, 1534, 47974, 6, 7896, 12, 10766, 2407, 131, 525, 2161, 225, 895, 6, 9840, 208, 10116, 16788, 5460, 6, 5363, 289, 10116, 26805, 1245, 6, 2224, 344, 10116, 5986, 6, 2995, 344, 10116, 6682, 6, 11026, 50118, 23301, 12, 698, 12, 3570, 50118, 19174, 100, 35, 698, 4, 1558, 5339, 73, 15545, 4, 642, 1264, 4, 151, 4956, 1225, 50118, 48407, 35, 7309, 12, 1409, 50118, 50118, 47638, 35, 23098, 45604, 35, 8133, 12, 560, 12, 14069, 9235, 36, 11674, 7164, 43, 16, 5, 1049, 1303, 9, 7947, 12, 134, 7910, 11, 408, 3612, 4, 6211, 14, 5, 230, 12, 12528, 25673, 179, 33915, 6, 385, 1397,

In [25]:
def preprocess_function(dataset):

  questions = [q.lstrip() for q in dataset["question"]]
  paragraphs = [p.lstrip() for p in dataset["context"]]

  tokenized_examples = tokenizer(
    questions,
    paragraphs,
    truncation="only_second",
    max_length=MAX_LENGTH,
    stride=64,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
  offset_mapping = tokenized_examples.pop("offset_mapping")

  tokenized_examples["start_positions"] = []#[152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  tokenized_examples["end_positions"] = []#[172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

  for i, offsets in enumerate(offset_mapping):
    sample_index = sample_mapping[i]

    start_char=dataset["answers"][sample_index]['answer_start'][0]
    end_char=start_char+len(dataset["answers"][sample_index]['text'][0])
    found=0
    start_token_position=0
    end_token_position=0

    for j,offset in enumerate(offsets):
      if offset[0]<=start_char and offset[1]>=start_char and found==0:
        start_token_position=j
        end_token_position=MAX_LENGTH
        found=1
      if offset[1]>=end_char and found==1:
        end_token_position=j
        break
    tokenized_examples["start_positions"].append(start_token_position)
    tokenized_examples["end_positions"].append(end_token_position)

  return tokenized_examples

In [26]:
tokenized_dataset=dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/2019 [00:00<?, ? examples/s]

In [27]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 35217
    })
})

In [28]:
tf_dataset = tokenized_dataset["train"].to_tf_dataset(
    shuffle=True,
    batch_size=BATCH_SIZE,
)

In [29]:
for i in tf_dataset.take(10):
  print(i)

{'input_ids': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[    0, 39487,  8489, ...,  1885,    90,     2],
       [    0, 32251,    32, ...,  2338,   131,     2],
       [    0,  7608,    21, ...,   289,   245,     2],
       ...,
       [    0,  2264,    21, ...,  2964,     4,     2],
       [    0,  2264,    21, ..., 41113,     9,     2],
       [    0,  2264,    21, ...,     4,  7655,     2]])>, 'attention_mask': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>, 'start_positions': <tf.Tensor: shape=(16,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>, 'end_positions': <tf.Tensor: shape=(16,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>}
{'input_ids': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[ 

In [30]:
train_dataset=tf_dataset.take(int(0.9*len(tf_dataset)))
val_dataset=tf_dataset.skip(int(0.9*len(tf_dataset)))

# **Modeling**

In [31]:
from transformers import LongformerTokenizer, TFLongformerForQuestionAnswering

model = TFLongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")

tf_model.h5:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFLongformerForQuestionAnswering.

All the layers of TFLongformerForQuestionAnswering were initialized from the model checkpoint at allenai/longformer-large-4096-finetuned-triviaqa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerForQuestionAnswering for predictions without further training.


In [32]:
model.summary()

Model: "tf_longformer_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 longformer (TFLongformerMa  multiple                  433551360 
 inLayer)                                                        
                                                                 
 qa_outputs (Dense)          multiple                  2050      
                                                                 
Total params: 433553410 (1.62 GB)
Trainable params: 433553410 (1.62 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
optimizer=Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer)

In [34]:
history=model.fit(train_dataset,validation_data=val_dataset,epochs=3)

Epoch 1/3


There should be exactly three separator tokens: 2 in every sample for questions answering. You might also consider to set `global_attention_mask` manually in the forward function to avoid this. This is most likely an error. The global attention is disabled for this forward pass.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


There should be exactly three separator tokens: 2 in every sample for questions answering. You might also consider to set `global_attention_mask` manually in the forward function to avoid this. This is most likely an error. The global attention is disabled for this forward pass.


ResourceExhaustedError: ignored

#Evaluation

In [ ]:
from evaluate import load

In [ ]:
squad_metric = load("squad")
predictions = [{'prediction_text': '1999', 'id': '56e10a3be3433e1400422b22'}]
references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
results = squad_metric.compute(predictions=predictions, references=references)
print(results)

#Testing

In [ ]:
question="How is the virus spread?"
text="We know that the disease is caused by the SARS-CoV-2 virus, which spreads between people in several different ways.Current evidence suggests that the virus spreads mainly between people who are in close contact with each other, for example at a conversational distance.The virus can spread from an infected person’s mouth or nose in small liquid particles when they cough, sneeze, speak, sing or breathe. Another person can then contract the virus when infectious particles that pass through the air are inhaled at short range (this is often called short-range aerosol or short-range airborne transmission) or if infectious particles come into direct contact with the eyes, nose, or mouth (droplet transmission). The virus can also spread in poorly ventilated and/or crowded indoor settings, where people tend to spend longer periods of time. This is because aerosols can remain suspended in the air or travel farther than conversational distance (this is often called long-range aerosol or long-range airborne transmission). People may also become infected when touching their eyes, nose or mouth after touching surfaces or objects that have been contaminated by the virus. Further research is ongoing to better understand the spread of the virus and which settings are most risky and why. Research is also under way to study virus variants that are emerging and why some are more transmissible. For updated information on SARS-CoV-2 variants, please read the weekly epidemiologic updates."
inputs = tokenizer(question, text, return_tensors="tf")
outputs = model(**inputs)

answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)